In [ ]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/LLM/few-shot'

/content/drive/MyDrive/LLM/few-shot


In [ ]:
!pip install torch transformers datasets sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 63.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [ ]:
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from sklearn.metrics import accuracy_score
import json

# 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
k_values = [0, 1, 2, 4, 8, 16]

# 데이터 로딩
train_set = load_from_disk('/content/drive/MyDrive/LLM/few-shot/data/train')
dev_set = load_from_disk('/content/drive/MyDrive/LLM/few-shot/data/dev')
test_set = load_from_disk('/content/drive/MyDrive/LLM/few-shot/data/test')

In [ ]:
train_set

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 800
})

In [ ]:
dev_set

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 100
})

In [ ]:
test_set

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 100
})

In [ ]:
train_list = list(train_set)

support_examples_dict = {}
for k in k_values:
  if k > 0:
    random.seed(42 + k)
    support_examples_dict[k] = random.sample(train_list, k)
  else:
    support_examples_dict[k] = []

In [ ]:
for k in k_values:
  print(support_examples_dict[k])

[]
[{'pubid': 14627582, 'question': 'Double reading of barium enemas: is it necessary?', 'context': {'contexts': ['The purpose of our study was to determine the effectiveness, clinical impact, and feasibility of double reading barium enemas.', "Independent double readings of 1,003 consecutive barium enemas (822 double- and 181 single-contrast examinations) were prospectively performed. From this pool of 1,003 examinations, 994 were included in our study. Examinations showing at least one polyp or carcinoma 5 mm or larger were considered to have positive results. For combined readings, results were considered positive if either of the two interpreters reported finding a polyp or carcinoma. A McNemar test was used to compare the first reader's results with the combined results of the first and second readers. Results were retrospectively correlated with endoscopic or surgical results in 360 patients, and agreement between first and combined readings and endoscopic results was determined.

In [ ]:
# 텍스트 생성 함수
def generate(model, tokenizer, prompt, max_new_tokens=64):
  inputs = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
  outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
  decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return decoded[len(prompt):].strip()

# 평가 함수
def evaluate(model, tokenizer, support_examples, test_set, result_file):
  preds, labels, results = [], [], []

  for example in test_set:
    prompt = "Examples:"
    for support in support_examples:
      prompt += f"Question: {support['question']}\nAnswer: {support['final_decision']}\n"

    prompt += "Answer based on the above examples"
    prompt += f"Question: {example['question']}\nAnswer: "
    prompt += "Answer only one word from the following options: yes, no, or maybe\nDo not provide any explanations.\nAnswer:"

    answer = generate(model, tokenizer, prompt)
    answer_lower = answer.lower().strip()

    # 정확한 매칭만 허용
    if answer_lower.startswith("yes"):
        pred = "yes"
    elif answer_lower.startswith("no"):
        pred = "no"
    elif answer_lower.startswith("maybe"):
        pred = "maybe"
    else:
        pred = 'error'  # fallback

    preds.append(pred)
    labels.append(example['final_decision'])
    results.append({
        'question': example['question'],
        'answer': answer,
        'prediction': pred,
        'label': example['final_decision']
    })

  acc = accuracy_score(labels, preds)

  with open(result_file, "w", encoding="utf-8") as f:
      json.dump(results, f, indent=2, ensure_ascii=False)

  return acc, preds, labels

In [ ]:
huggingface_model_id = "Qwen/Qwen3-1.7B"

print("\n==== Evaluating Original Hugging Face Qwen Model ====")
model = AutoModelForCausalLM.from_pretrained(huggingface_model_id, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(huggingface_model_id, trust_remote_code=True)
model.eval()

for k in k_values:
	support_examples = support_examples_dict[k]
	result_file = f"results_qwen_k{k}.json"
	print(f"Evaluating model {k} shots...")
	acc, preds, labels = evaluate(model, tokenizer, support_examples, test_set, result_file)
	print(f"[Original Qwen (HF)] Accuracy: {acc:.4f}")


==== Evaluating Original Hugging Face Qwen Model ====


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating model 0 shots...
[Original Qwen (HF)] Accuracy: 0.5600
Evaluating model 1 shots...
[Original Qwen (HF)] Accuracy: 0.3600
Evaluating model 2 shots...
[Original Qwen (HF)] Accuracy: 0.4200
Evaluating model 4 shots...
[Original Qwen (HF)] Accuracy: 0.4700
Evaluating model 8 shots...
[Original Qwen (HF)] Accuracy: 0.4100
Evaluating model 16 shots...
[Original Qwen (HF)] Accuracy: 0.4600


In [ ]:
# IA3 SFT 모델 평가
ia3_model_path = "/content/drive/MyDrive/LLM/few-shot/models/qwen_pubmedqa_ia3_sft/final"

print("\n==== Evaluating IA3 SFT Model ====")
model = AutoModelForCausalLM.from_pretrained(ia3_model_path, trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(ia3_model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model.eval()

for k in k_values:
	support_examples = support_examples_dict[k]
	result_file = f"results_ia3_k{k}.json"
	print(f"Evaluating model {k} shots...")
	acc, preds, labels = evaluate(model, tokenizer, support_examples, test_set, result_file)
	print(f"[AI3 Qwen] Accuracy: {acc:.4f}")


==== Evaluating IA3 SFT Model ====


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating model 0 shots...
[AI3 Qwen] Accuracy: 0.5100
Evaluating model 1 shots...
[AI3 Qwen] Accuracy: 0.3600
Evaluating model 2 shots...
[AI3 Qwen] Accuracy: 0.3800
Evaluating model 4 shots...
[AI3 Qwen] Accuracy: 0.4400
Evaluating model 8 shots...
[AI3 Qwen] Accuracy: 0.4200
Evaluating model 16 shots...
[AI3 Qwen] Accuracy: 0.5000


In [ ]:
metaicl_base_path = "/content/drive/MyDrive/LLM/few-shot/models/metaicl_qwen_meta"

for model_k in k_values:
    print(f"\n==== Loading MetaICL Model trained with k={model_k} shots ====")
    model_path = f"{metaicl_base_path}/k_{model_k}/final"
    model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    model.eval()

    for eval_k in k_values:
        support_examples = support_examples_dict[eval_k]
        result_file = f"results_metaicl_model_k{model_k}_eval_k{eval_k}.json"
        print(f"Evaluating model trained on k={model_k} shots with support set k={eval_k}...")
        acc, preds, labels = evaluate(model, tokenizer, support_examples, test_set, result_file)
        print(f"[MetaICL Model k={model_k}] Accuracy with support k={eval_k}: {acc:.4f}")


==== Loading MetaICL Model trained with k=0 shots ====


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Evaluating model trained on k=4 shots with support set k=0...
[MetaICL Model k=4] Accuracy with support k=0: 0.6000
Evaluating model trained on k=4 shots with support set k=1...
[MetaICL Model k=4] Accuracy with support k=1: 0.3600
Evaluating model trained on k=4 shots with support set k=2...
[MetaICL Model k=4] Accuracy with support k=2: 0.4300
Evaluating model trained on k=4 shots with support set k=4...
[MetaICL Model k=4] Accuracy with support k=4: 0.5100
Evaluating model trained on k=4 shots with support set k=8...
[MetaICL Model k=4] Accuracy with support k=8: 0.4200
Evaluating model trained on k=4 shots with support set k=16...
[MetaICL Model k=4] Accuracy with support k=16: 0.4300

==== Loading MetaICL Model trained with k=8 shots ====


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating model trained on k=8 shots with support set k=0...
[MetaICL Model k=8] Accuracy with support k=0: 0.5000
Evaluating model trained on k=8 shots with support set k=1...
[MetaICL Model k=8] Accuracy with support k=1: 0.3700
Evaluating model trained on k=8 shots with support set k=2...
[MetaICL Model k=8] Accuracy with support k=2: 0.4100
Evaluating model trained on k=8 shots with support set k=4...
[MetaICL Model k=8] Accuracy with support k=4: 0.5400
Evaluating model trained on k=8 shots with support set k=8...
[MetaICL Model k=8] Accuracy with support k=8: 0.3800
Evaluating model trained on k=8 shots with support set k=16...
[MetaICL Model k=8] Accuracy with support k=16: 0.4800

==== Loading MetaICL Model trained with k=16 shots ====


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating model trained on k=16 shots with support set k=0...
[MetaICL Model k=16] Accuracy with support k=0: 0.5000
Evaluating model trained on k=16 shots with support set k=1...
[MetaICL Model k=16] Accuracy with support k=1: 0.3400
Evaluating model trained on k=16 shots with support set k=2...
[MetaICL Model k=16] Accuracy with support k=2: 0.3900
Evaluating model trained on k=16 shots with support set k=4...
[MetaICL Model k=16] Accuracy with support k=4: 0.4500
Evaluating model trained on k=16 shots with support set k=8...
[MetaICL Model k=16] Accuracy with support k=8: 0.3900
Evaluating model trained on k=16 shots with support set k=16...
[MetaICL Model k=16] Accuracy with support k=16: 0.4700
